In [37]:
import numpy as np
import tensorflow as tf
import cv2
import pathlib
from PIL import Image
import glob

# model path 
model_path = "model/trained.tflite"
# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path=model_path)
# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

interpreter.allocate_tensors()

# input details
print(input_details)


[{'name': 'serving_default_x:0', 'index': 0, 'shape': array([ 1, 23, 23,  3], dtype=int32), 'shape_signature': array([ 1, 23, 23,  3], dtype=int32), 'dtype': <class 'numpy.int8'>, 'quantization': (0.0038446751423180103, -128), 'quantization_parameters': {'scales': array([0.00384468], dtype=float32), 'zero_points': array([-128], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]


In [50]:
def infer_with_TF_lite(interpreter, input_details, output_details, raw_image):
    # Get input size
    input_shape = input_details[0]['shape']
    size = input_shape[:2] if len(input_shape) == 3 else input_shape[1:3]
    print(size)

    # Preprocess image
    raw_image = raw_image.resize(size)
#     img = np.array(raw_image, dtype=np.float32)
    img = np.array(raw_image)

    # Normalize image
#     img = img / 255.

    # Add a batch dimension and a dimension because we use grayscale format
    # Reshape from (INPUT_IMAGE_SIZE, INPUT_IMAGE_SIZE) to (1, INPUT_IMAGE_SIZE, INPUT_IMAGE_SIZE, 1)
    #input_data = img.reshape(1, img.shape[0], img.shape[1], 3)
    input_data = img.reshape((-1, 1, 23, 23))

    
    # Point the data to be used for testing
    interpreter.set_tensor(input_details[0]['index'], input_data)

    # Run the interpreter
    interpreter.invoke()

    # Obtain results
    predictions = interpreter.get_tensor(output_details[0]['index'])[0]
    
    return predictions

In [51]:
path = "sample/"
file_format = ".png"
files = [files for files in glob.glob(path + "*" + file_format, recursive=False)]
print("Found {0} {1} files".format(len(files), file_format))


Found 51 .png files


In [52]:
for f in files:
    # Read image
    raw_image = Image.open(f).convert('L')
    print(raw_image)
    result_tfl = infer_with_TF_lite(interpreter, input_details, output_details, raw_image)

    result_mcu = infer_with_MCU(raw_image)

    status = compare_results(result_tfl, result_mcu)
    status_str = "Ok" if status else "Reults NOT eqal!"
    print("For file: {0} - {1}".format(f, status_str))

<PIL.Image.Image image mode=L size=15x25 at 0x7FBB78733730>
[23 23]


ValueError: Cannot set tensor: Got value of type UINT8 but expected type INT8 for input 0, name: serving_default_x:0 

In [8]:
image_transform = cv2.imread("1.bmp", cv2.IMREAD_GRAYSCALE)
print(image_transform)
# plt.imshow(image_transform)
# plt.show()

None
